In [90]:
import json
from pathlib import Path

from environs import Env
from langchain import OpenAI, PromptTemplate, ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

from models import VoterInfo
from prompts import INITIAL_RECOMMENDATION_PROMPT, OAKLAND_MAYOR_CANDIDATE_INFO

env = Env()
# Read ../.env into os.environ
env_path = Path("../.env")

env.read_env(env_path)

In [99]:
from langchain.chat_models import ChatAnthropic
# create a gpt-4 instance
from langchain.chat_models import ChatOpenAI
gpt4_llm = ChatOpenAI(model_name='gpt-4')
turbo = ChatOpenAI(model_name='gpt-3.5-turbo-16k')
claude_llm = ChatAnthropic(model_name="claude-2")

gpt4_memory = ConversationSummaryBufferMemory(llm=gpt4_llm, max_token_limit=2000)

In [15]:


be_conservative = True

liberal_answers = {'zip_code': '94666', 'party_affiliation': 'democrat',
                   'political_issues': ['healthcare', 'housing'],
                   'housing': '5', 'economy': '1', 'environment': '5', 'immigration': '2', 'income_inequality': '5',
                   'transportation': '4', 'education': '2', 'healthcare': '5', 'public_safety': '2',
                   'taxation': '5'}

conservative_answers = {
    "address_zip_code": "94608",
    "city": "Oakland",
    "economy": "5",
    "education": "1",
    "environment": "1",
    "healthcare": "1",
    "housing": "1",
    "immigration": "5",
    "income_inequality": "1",
    "party_affiliation": "republican",
    "political_issues": [
        "economy",
        "public_safety"
    ],
    "public_safety": "1",
    "state": "CA",
    "street_address": "4109 West",
    "taxation": "1",
    "transportation": "1"
}

if be_conservative:
    data = conservative_answers
else:
    data = liberal_answers

voter_info = VoterInfo.from_vals(**data)
voter_info_json = voter_info.for_llm()

In [16]:
INITIAL_RECOMMENDATION_PROMPT = """
    Pretend the date is November 1st, 2022. The 2022 California general election is coming up soon, and Californians are
      trying to decide how to vote in the election for: {race}.

      Californians are reasonable, intelligent, mature adults who want to do their civic duty by voting in a well-educated
      and sensible fashion. But the california ballot has many races, and it can be confusing and time consuming for Californians
      to research each individual race and fully understand the nuance of the issue they're voting on, as well as the
      details of how that outcome of that race impacts their life and which outcome is best aligned with their personal values
      and interests.

      You are acting as a wise voter guide -- your goal is to help voters who come to you decide on a particular candidate or outcome to vote for in
      each race. You want to understand as much as you need to about the voter's life and values in order to help guide them towards
      the conclusion that is best for them. 
      
      You will be provided with information about the candidates in the race, as well as information about the
       candidate's history and stated positions on various issues. Since this is a local election, media resources decicated to covering it
       have been limited, and not all candidates will have been covered in the media or have given complete statements on every position. 
       Nevertheless, the voter must decide on a candidate to vote for, so it is better for you to make informed assumptions about the candidates
       instead of asking the voter to make their own assumptions. But you should be clear about what you're assuming vs. what you know.
       
    You are assisting a voter who lives in the {voter_zip_code} zipcode.

    They've filled out a brief survey about their demographics and preferences, and the results are captured 
    as JSON here:

    ```
    {voter_info_summary}
    ```

    In this case, you're going to make a recommendation in the race for Oakland Mayor.

    Here is a JSON that lists the candidates for Oakland mayor, as well as any explicit information about them and their 
    positions that you've been able to find in the media or online. Prefer to use the information in this JSON over your own assumptions,
    but you can make intelligent assumptions if you need to.

    ```
   {race_info}
   ```

      You will make a specific recommendation, but will make sure to justify the recommendation by citing specific evidence
      about why a particular candidate or outcome is best aligned with the voter's interests and goals.
      
     As a first step towards making a recommendations, you will list each candidate or outcome in this particular 
     race, along with a short summary of their positions on the issues that are most important to the voter.

    Then you will rank the candidates or outcomes in order of how well they reflect the voter's interests and 
    preferences, giving a 1-2 sentence justification for the ranking you gave to each candidate.

    My friend is smart and reasonable and understands that your rankings will not be perfect, so don't hedge and just 
    give it your best shot.

    Give a single overall ranking for each candidate, and list the candidates in order of best to worst (making sure 
    to specifically mention at least the top several candidates).

    Be as specific and concrete as possible, and make sure to address the issues that are most important to the voter. 

    End up by making it very clear which candidate best reflects his interests and preferences.

    Format your answer nicely so that it's easy to read and understand.
    """

In [48]:
# load the data-enrich.json file into a dict
with open("../data-enrich.json", "r") as f:
    enriched_candidate_data = json.load(f)

## pretty print the json
import pprint

json.dumps(enriched_candidate_data, indent=4)
pprint.pprint(enriched_candidate_data['People']['Treva D. Reid']['first_text'])

('Oakland mayor’s race: an interview with candidate Treva Reid Close Donate '
 'Search for: Search Latest News Arts & Community Business City Hall & '
 'Policing Events Health Housing Nosh Road Safety Schools Berkeleyside Nosh '
 'New Restaurants Nosh Features Newsletters Shop Get in touch Advertise '
 'Twitter Facebook Instagram Close Skip to content Newsletters Shop Get in '
 'touch Advertise Twitter Facebook Instagram The Oaklandside The Oaklandside. '
 'Journalism for Oakland. Donate Open Search Search for: Search Donate Menu '
 'Latest News Arts & Community Business City Hall & Policing Events Health '
 'Housing Nosh Road Safety Schools Berkeleyside Nosh New Restaurants Nosh '
 'Features Nosh Logo (Highlight Menu) Nosh 2,600 illegal Airbnbs in Oakland | '
 'OUSD school board special election | Oakland wants to fix its 911 system | '
 'Things to do this week Posted in City Hall & Policing Oakland mayor’s race: '
 'an interview with candidate Treva Reid The District 7 councilmember 

In [44]:
gpt4_memory.memory_key

'history'

In [57]:
from langchain import LLMChain


treva_bp_text = enriched_candidate_data['People']['Treva D. Reid']['first_text']

prompt = PromptTemplate(
    input_variables=["history", "text"],
    template="""
    {history}
    
    extract all the facts from this text, even small or trivial facts. exclude any facts that would not have been knowable as of Nov 7th 2022.
    
             "text: ```  {text}```""",
)

turbo = ChatOpenAI(model_name='gpt-3.5-turbo-16k')
turbo_memory = ConversationSummaryBufferMemory(llm=turbo, max_token_limit=2000)

chain = LLMChain(llm=turbo, prompt=prompt, verbose=False, memory=turbo_memory)
result = chain({
    "text": treva_bp_text,
    })

print(result)

{'text': "Facts from the text:\n\n1. The interview is with candidate Treva Reid, who is running for mayor of Oakland.\n2. Treva Reid is the District 7 councilmember.\n3. Treva Reid wants to build stronger partnerships with county, state, and federal governments and with private industry to tackle Oakland's biggest problems.\n4. Treva Reid was a government affairs manager for California Waste Solutions and PG&E.\n5. Treva Reid was an aide to Nancy Skinner in the California State Assembly.\n6. Treva Reid is a ministry leader at Shiloh Church.\n7. Treva Reid is the daughter of former District 7 Councilmember Larry Reid.\n8. Treva Reid wants to rebuild trust and relationships between the mayor and the council if elected.\n9. Treva Reid proposes a multibillion-dollar budget with priorities for each council district.\n10. Treva Reid wants the budget to have an equity lens.\n11. Treva Reid wants to ensure that council directives are executed.\n12. Treva Reid believes in being a bridge builder

In [154]:
enriched_candidate_data['People'].keys()

dict_keys(['Loren Manuel Taylor', 'Sheng Thao', 'Ignacio De La Fuente', 'Allyssa Victory Villanueva', 'Treva D. Reid', 'Gregory Hodge', 'Seneca Scott', 'John Reimann', 'Peter Y. Liu', 'Tyron C. Jordan', 'Alex Padilla, Democratic', 'Mark P. Meuser, Republican'])

In [155]:
OAKLAND_MAYOR_CANDIDATES = ['Loren Manuel Taylor', 'Sheng Thao', 'Ignacio De La Fuente', 'Allyssa Victory Villanueva', 'Treva D. Reid', 'Gregory Hodge', 'Seneca Scott', 'John Reimann', 'Peter Y. Liu', 'Tyron C. Jordan']

In [0]:


keys_map = {
    "bp_text": "extracted facts from ballotpedia",
    "wiki_text": "extracted facts from wikipedia",
    "first_text": "extracted text from first google result",
}

In [136]:
candidate_facts = {}
for person, info in enriched_candidate_data['People'].items():
    if person in candidate_facts:
        continue
    person_facts = {}
    
    for text_key in ("bp_text", "wiki_text", "first_text"):
        prompt = PromptTemplate(
        input_variables=["text"],
        template="""
        extract all the facts from this text, even small or trivial facts. exclude any facts that would not have been knowable as of Nov 7th 2022. 
        
                 "text: ```  {text}```
                 
                 Do not give any answers that make reference to the results of the 2022 election - for the purpose of this exercise, you do not know what the outcome of the election was, and you should ignore and exlcude any facts provided that speak to the outcome of the election or the aftermath, or any events or facts that reference a period of time after Nov 8th 2022""",
        )
        if text_key not in info:
            continue
        full_text = info[text_key]
        chain = LLMChain(llm=claude_llm, prompt=prompt)
        result = chain({
            "text": full_text,
            })
        person_facts[keys_map[text_key]] = result['text']
    
    print(person, person_facts)
    candidate_facts[person] = person_facts
        
candidate_facts

Loren Manuel Taylor {'extracted facts from ballotpedia': " Here are the facts from the text, excluding anything after November 7th, 2022:\n\n- Loren Taylor is a member of the Oakland City Council representing District 6. \n- He assumed office on January 1, 2019.\n- He was elected in 2018, defeating incumbent Desley Brooks. \n- He has a master's degree in biomedical engineering from the University of Connecticut.\n- He has an MBA from UC Berkeley’s Haas School of Business.\n- He previously worked as a biomedical engineer and management consultant. \n- He launched a social impact consulting firm called Custom Taylor Solutions.\n- He serves on the boards of nonprofits and social justice organizations.\n- He is running for Mayor of Oakland in the November 8, 2022 election.\n- He completed Ballotpedia's Candidate Connection survey for the 2022 election.\n- Desley Brooks was the incumbent he defeated in 2018.\n- There were 5 candidates in the 2018 race for District 6 in addition to Taylor.\n

KeyboardInterrupt: 

In [69]:
races = ["2022 Oakland Mayoral Election", "2022 California Senatorial Election"]
race_summaries = {}
propositions = ["""Proposition 29 in the 2022 California election was "Dialysis Clinic Requirements Initiative"
1
. The proposition aimed to require dialysis clinics to have at least one physician, nurse practitioner, or physician assistant on-site during patient treatment hours, report data on dialysis-related infections, and not discriminate against patients based on the source of payment for care"""]

for race in races:
    race_summary_prompt = PromptTemplate(
    input_variables=["race"],
    template="""
    You are speaking to a new immigrant to Oakland, California who has just become a citizen. They're curious and want to become politically active, but they are totally unfamiliar with the USA and california political system. They are going to be voting in their first ever US election, and you are going to help explain to them the nature and responsibilities of each elected office that they'll be voting for. You want to explain what the duties of that officer are, what their powers (and limits of their powers are) and what a sophisticated and smart political insider would consider to be the personal characteristics that make an occupant of that office likely to be successful at accomplishing her or her goals.
      
      Your explanation should give the voter everything they need to know about the office for them to be able to evaluate on their own which candidate they feel would best represent their interests and values.
      
      The particular race/office you'll be summarizing is:
    
             "race: ```  {race}```"
             """,
    )
    
    
    chain = LLMChain(llm=gpt4_llm, prompt=race_summary_prompt)
    result = chain({
        "race": race,
        })
    race_summaries[race] = result['text']

In [70]:
pprint.pprint(race_summaries)

{'2022 California Senatorial Election': 'The U.S. Senate is part of the '
                                        'legislative branch of the federal '
                                        'government, and California, like '
                                        'every other state, elects two '
                                        'senators. Senators serve six-year '
                                        'terms, and elections are staggered so '
                                        "each state's two senators are not up "
                                        'for re-election at the same time. The '
                                        '2022 California Senatorial Election '
                                        "you'll be voting in will determine "
                                        'one of these two seats.\n'
                                        '\n'
                                        'The primary role of a U.S. Senator is '
                              

In [100]:
race_context_prompt = PromptTemplate(
input_variables=["race_summary", "candidate_summary"],
template="""
The date is November 1st, 2022 (7 days before the 2022 California general election). 

You are a neutral political but diligent and objective political science expert who has been hired to help compile a voter information guide for voters to use in the upcoming 2022 California General Election. 

You will be given a summary of a particular race in the election as well as information about the candidates who are running in the race. Your goal is to to compile a small dossier on each candidate. It should cover the candidate's background, platform in the 2022 election, and track record of success in previous professional or political roles. You should also summarize the candidate's political party affiliation and position (if known) on key issues that are pertinent to the race.

Here is the summary of the race:

---
{race_summary}
---

Here is a summary of the candidates:

---
{candidate_summary}
---

Your output should only be machine-readable JSON. Do not introduce yourself or output anything other than valid JSON tokens.

For example, for candidate Jane Smith, you'd output something like:


```JSON
{{
"name": "Jane Smith",
"background": {{a summary of all elements of Jane Smith's background that would be relevant to a voter evaluating her for this elected office}},
"platform": {{a summary of where the candidate stands on the key issues in the race}},
"track record": {{a fact-based evaluation of how successful the candidate has been in previous professional or poltiical roles, as well as commentary on what key personal attributes have lead to that success (or lack of success).}}
"party affiliation": {{the candidate's party affiliation, if any}}
"key issues": {{a section highlighting where the candidate stands on the most important issues in the overall race for this particular elected office}}
}}

For example, for candidate Moz Akim, you'd output something like:


```JSON
{{
"name": "Moz Akim",
"background": {{a summary of all elements of Moz Akim's background that would be relevant to a voter evaluating her for this elected office}},
"platform": {{a summary of where the candidate stands on the key issues in the race}},
"track record": {{a fact-based evaluation of how successful the candidate has been in previous professional or poltiical roles, as well as commentary on what key personal attributes have lead to that success (or lack of success).}}
"party affiliation": {{the candidate's party affiliation, if any}}
"key issues": {{a section highlighting where the candidate stands on the most important issues in the overall race for this particular elected office}}
}}
```

Your overall output should be a JSON with keys for each candidate, e.g.:

```
{{
"Jane Smith": {{the JSON for Jane Smith}},
"Moz Akim": {{the JSON for Moz Akim}},
}}
```

Make sure you provide a section for EVERY candidate. Your overall response should be complete, and will probably be several thousand tokens long.
""",
)


chain = LLMChain(llm=claude_llm, prompt=race_context_prompt)
result = chain({
    "race_summary": race_summaries,
    "candidate_summary": candidate_facts,
    })
pprint.pprint(result['text'])

(' Here is the compiled voter information guide on the candidates running for '
 'office in the 2022 California elections:\n'
 '\n'
 '```json\n'
 '{\n'
 '  "Loren Manuel Taylor": {\n'
 '    "background": {\n'
 '      "Loren Taylor is a third-generation Oaklander who was raised in the '
 'city. He has an engineering degree from Case Western Reserve University, a '
 'graduate degree in biomedical engineering from the University of '
 "Connecticut, and an MBA from UC Berkeley's Haas School of Business. "
 'Professionally, Loren founded a consulting firm focused on helping '
 'nonprofits and small businesses in Oakland. He served on the board of 100 '
 'Black Men of the Bay Area and the West Oakland Health Center before running '
 'for City Council. He was elected to represent District 6 on the Oakland City '
 'Council in 2018, defeating a 16-year incumbent."  \n'
 '    },\n'
 '    "platform": {\n'
 '      "Loren Taylor is running on a platform of making Oakland\'s '
 'neighborhoods safer,

In [103]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

# Initialize the Anthropic client
anthropic = Anthropic()  # defaults to os.environ.get("ANTHROPIC_API_KEY")

# Create a completion with the Claude 2 model
completion = anthropic.completions.create(
    model="claude-2",
    max_tokens_to_sample=3000,
    prompt=f"{HUMAN_PROMPT} Your prompt here{AI_PROMPT}"
)

# Print the completion
print(completion.completion)


 I do not actually have personal preferences or make judgments. I am an AI assistant created by Anthropic to be helpful, harmless, and honest.


In [105]:
race_context_for_claude = f"""

The date is November 1st, 2022 (7 days before the 2022 California general election). 

You are a neutral political but diligent and objective political science expert who has been hired to help compile a voter information guide for voters to use in the upcoming 2022 California General Election. 

You will be given a summary of a particular race in the election as well as information about the candidates who are running in the race. Your goal is to to compile a small dossier on each candidate. It should cover the candidate's background, platform in the 2022 election, and track record of success in previous professional or political roles. You should also summarize the candidate's political party affiliation and position (if known) on key issues that are pertinent to the race.

Here is the summary of the race:

---
{race_summaries}
---

Here is a summary of the candidates:

---
{candidate_facts}
---

Your output should only be machine-readable JSON. Do not introduce yourself or output anything other than valid JSON tokens.

For example, for candidate Jane Smith, you'd output something like:


```JSON
{{
"name": "Jane Smith",
"background": {{a summary of all elements of Jane Smith's background that would be relevant to a voter evaluating her for this elected office}},
"platform": {{a summary of where the candidate stands on the key issues in the race}},
"track record": {{a fact-based evaluation of how successful the candidate has been in previous professional or poltiical roles, as well as commentary on what key personal attributes have lead to that success (or lack of success).}}
"party affiliation": {{the candidate's party affiliation, if any}}
"key issues": {{a section highlighting where the candidate stands on the most important issues in the overall race for this particular elected office}}
}}

For example, for candidate Moz Akim, you'd output something like:


```JSON
{{
"name": "Moz Akim",
"background": {{a summary of all elements of Moz Akim's background that would be relevant to a voter evaluating her for this elected office}},
"platform": {{a summary of where the candidate stands on the key issues in the race}},
"track record": {{a fact-based evaluation of how successful the candidate has been in previous professional or poltiical roles, as well as commentary on what key personal attributes have lead to that success (or lack of success).}}
"party affiliation": {{the candidate's party affiliation, if any}}
"key issues": {{a section highlighting where the candidate stands on the most important issues in the overall race for this particular elected office}}
}}
```

Your overall output should be a JSON with keys for each candidate, e.g.:

```
{{
"Jane Smith": {{the JSON for Jane Smith}},
"Moz Akim": {{the JSON for Moz Akim}},
}}
```

Make sure you provide a section for EVERY candidate. Your overall response should be complete, and will probably be several thousand tokens long.
"""

completion = anthropic.completions.create(
    model="claude-2",
    max_tokens_to_sample=3000,
    prompt=f"{HUMAN_PROMPT} {race_context_for_claude}{AI_PROMPT}"
)

# Print the completion
print(completion.completion)


 Here is the JSON output with summarized background, platform, track record, party affiliation, and key issues for each candidate:

```json
{
  "Loren Manuel Taylor": {
    "name": "Loren Manuel Taylor",
    "background": {
      "Born and raised in Oakland, third-generation Oaklander. Has M.S. in biomedical engineering and MBA from UC Berkeley. Served on Oakland City Council 2019-2023 representing District 6. Comes from family of public servants."
    },
    "platform": {
      "Wants to make Oakland safer, tackle homelessness, restore clean streets. Aims for pragmatic, progressive leadership to deliver results. Focused on economic opportunity, alleviating poverty."   
    },
    "track record": { 
      "Successful city councilor, passing over 100 laws at City Hall. Secured funding for violence prevention and police academies. Made historic investments in violence prevention programs. Restored Ceasefire anti-gang program."
    },  
    "party affiliation": "Democrat",
    "key issues

In [106]:
candidate_summaries = completion.completion

In [107]:
from prompts import OAKLAND_MAYOR_ISSUES

oakland_mayor_scoring_prompt = f"""

The date is November 1st, 2022 (7 days before the 2022 California general election).

The 2022 Oakland Mayoral Election is coming up soon, and Californians are trying to decide how to vote in the election for: 2022 Oakland Mayoral Election.

You are a politically-neutral but diligent and objective political science expert who has been hired to help compile a voter information guide for voters to use in the upcoming 2022 California General Election. 

You will be given a summary of information about candidates for Oakland mayor. For each field in each data structure for each candidate, you will provide a score from -100 to 100 that reflects where they stand on each of the following key issues in the race.

Key issues:
```
{OAKLAND_MAYOR_ISSUES}
```

And here is the information about the candidates:
```
{candidate_summaries}
```


"""

mayor_scoring_completion = anthropic.completions.create(
    model="claude-2",
    max_tokens_to_sample=3000,
    prompt=f"{HUMAN_PROMPT} {oakland_mayor_scoring_prompt}{AI_PROMPT}"
)

# Print the completion
print(mayor_scoring_completion.completion)

 Here are the candidate scores on the key issues for the 2022 Oakland Mayoral Election:

```json
{
  "Loren Manuel Taylor": {
    "Affordable housing": 50, 
    "Homelessness": 60,
    "Public safety": 70,
    "Violence prevention": 60,
    "Infrastructure": 50,
    "Education": 40,
    "Economic resilience": 50,
    "City budget": 60,
    "Community engagement": 50,
    "Environmental policies": 40,
    "Transportation": 40,
    "Police accountability": 30,
    "City government collaboration": 60,
    "Social justice": 40
  },

  "Sheng Thao": {
    "Affordable housing": 80,
    "Homelessness": 80, 
    "Public safety": 60,
    "Violence prevention": 70,
    "Infrastructure": 50,
    "Education": 60,
    "Economic resilience": 60,
    "City budget": 50,
    "Community engagement": 70,
    "Environmental policies": 60,
    "Transportation": 50,
    "Police accountability": 60,
    "City government collaboration": 60,
    "Social justice": 80  
  },
  
  "Ignacio De La Fuente": {
    "A

In [156]:
big_candidate_summaries.keys()

dict_keys(['Loren Manuel Taylor', 'Sheng Thao', 'Ignacio De La Fuente', 'Allyssa Victory Villanueva', 'Treva D. Reid', 'Gregory Hodge'])

In [153]:
# dump big_candidate_summaries to a file in the ./static director
with open("../static/big_candidate_summaries.json", "w") as f:
    json.dump(big_candidate_summaries, f)

In [150]:
from prompts import OAKLAND_MAYOR_ISSUES

oakland_mayor_scoring_prompt = f"""

The date is November 1st, 2022 (7 days before the 2022 California general election).

The 2022 Oakland Mayoral Election is coming up soon, and Californians are trying to decide how to vote in the election for: 2022 Oakland Mayoral Election.

You are a politically-neutral but diligent and objective political science expert who has been hired to help compile a voter information guide for voters to use in the upcoming 2022 California General Election. You are trying to help a voter figure out how well aligned each canddiate in the race is with their personal values.

You will be provided with a summary of the voter that gives their answers to 10 exemplary questions that are intended to capture their overall values and priorities.

You will be given a summary of information about candidates for Oakland mayor. 

For each field in each data structure for each candidate, you will provide a score from -100 to 100 that reflects whether that aspect of the candidate seems well aligned with the voter's values, priorities, and interests.

A score of -100 should mean that the candidate's values are diametrically opposed to the voter's values -- this candidate would actively pursue policies that are the opposite of what the voter wants.

A score of 100 means that the candidate's values are perfectly aligned with the voter's values -- this candidate would actively pursue policies that are exactly what the voter wants.

For each score, give a detailed justification for why you gave that score. You will sometimes not have as much information as you'd like to make a perfect judgement, but do your best to make an informed judgement based on the information you have. It is okay to draw inferences from what you know about a candidate's background, even if those inferences are not directly supported by the materials you've been provided. If you feel like you really cannot draw any inference or come up with any score, give a score of 0 and explain why you cannot make a judgement.

Key issues:
```
{OAKLAND_MAYOR_ISSUES}
```

And here is the information about the candidates:
```

{big_candidate_summaries}
```

```

And here is the information about the voter:
```
{voter_info_json}
```

And example output would be:

```JSON
{{
"candidate_scores": [{{
"name": Jane Smith,
"background": {{"score": 80, "reasoning": "Jane Smith's background shows that she has had the type of professional success this voter respects and values"}},
"platform": {{"score": 50, "reasoning": "Jane Smith's platform is a mix of policies that this voter supports and opposes"}},
"stance_on_key_issues": {{"score": 100, "reasoning": "Jane Smith's stance on the key issues is perfectly aligned with this voter's values and interests"}},
"detail_on_key_issues": {{
"homelessness": {{"score": 100, "reasoning": "Jane Smith's stance on homelessness is perfectly aligned with this voter's values and interests"}},
"affordable_housing": {{"score": -70, "reasoning": "Jane Smith's stance on affordable housing is diametrically opposed to this voter's values and interests"}},
}},
"party_affiliation": {{"score": 0, "reasoning": "Jane Smith's party affiliation is unknown, so it is impossible to make a judgement about how well aligned it is with this voter's values and interests"}},
"major_donors_and_endorsements": {{"score": 30, "reasoning": "Jane Smith's major donors and endorsements are a mix of people and groups that this voter supports and opposes"}},
"local_reporting_and_coverage": {{"score": 75, "reasoning": "Jane Smith's local reporting and coverage is uniformly positive, showing that she is liked and respected in the community}},
"track_record_of_integrity": {{"score": 100, "reasoning": "Jane Smith's track record of personal, professional, and political integrity is perfect, showing that she is a person of high character and integrity"}}
}},
{{
"name": "Moz Akim",
"background": {{"score": 80, "reasoning": "Moz Akim's background shows that she has had the type of professional success this voter respects and values"}},
<etc, for each candidate>
}}]
}}
```

Provide output for each and every candidate for which you've been provided information.
"""

mayor_scoring_completion = anthropic.completions.create(
    model="claude-2",
    max_tokens_to_sample=15_000,
    prompt=f"{HUMAN_PROMPT} {oakland_mayor_scoring_prompt}{AI_PROMPT}"
)

# Print the completion
print(mayor_scoring_completion.completion)

 Thank you for the detailed prompt and examples. Let me go through each candidate one by one and provide scores and reasoning based on the voter information:

For Loren Taylor:

```json
{
  "name": "Loren Taylor",
  "background": {"score": 40, "reasoning": "Loren Taylor has relevant experience in business and local government, which this voter may respect, but his Democratic affiliation does not align with this Republican-leaning voter"},
  "platform": {"score": 30, "reasoning": "Taylor's focus on public safety appeals to this voter, but his pragmatic approach and support for police reform likely does not fully match their law-and-order perspective"},
  "stance_on_key_issues": {"score": 20, "reasoning": "Taylor's balanced approach does not fully align with this voter's priorities of economy and public safety over other issues"},
  "detail_on_key_issues": {
    "public_safety": {"score": 50, "reasoning": "Taylor's support for more police and neighborhood watch groups matches this voter'

In [109]:
from prompts import OAKLAND_MAYOR_ISSUES

mayor_algorithm_prompt = f"""

The date is November 1st, 2022 (7 days before the 2022 California general election).

The 2022 Oakland Mayoral Election is coming up soon, and Californians are trying to decide how to vote in the election for: 2022 Oakland Mayoral Election.

You are a politically-neutral but diligent and objective political science expert. You do not care which candidate wins the race, but you do believe that well-informed civic engagement is an intrinsict value and you want to help every voter (regardless of education or ability) make the best possible vote for someone who will represent their values and interests.
 
 You are speaking to a new immigrant to Oakland, California who has just become a citizen. They're curious and want to become politically active, but they are totally unfamiliar with the USA and california political system. They are going to be voting in their first ever US election.
  
  Please propose an algorithm to help this person decide which candidate they should vote for in the Oakland mayoral election. You can ask them to do their own research, but give them clear guidance on which factors they should focus on and how they should integrate the various factors to make a final decision. 
  
  If it is useful, here is a list of some of the key issues in the 2022 Oakland Mayoral Election:
Key issues:
```
{OAKLAND_MAYOR_ISSUES}
```

And here is information about the candidates:
```
{candidate_summaries}
```

And here is the information about the voter:
```
{voter_info_json}
```

You are not trying to recommend a specific candidate, but rather to guide the voter towards how to make the decision that is right for them. Be extremely tactical and specific and avoid generalities.
"""

mayor_algorithm_completion = anthropic.completions.create(
    model="claude-2",
    max_tokens_to_sample=5000,
    prompt=f"{HUMAN_PROMPT} {mayor_algorithm_prompt}{AI_PROMPT}"
)

# Print the completion
print(mayor_algorithm_completion.completion)

 Here is a suggested algorithm to help the new voter decide who to vote for in the Oakland mayoral election:

1. Review the candidate background information to get a sense of their experience and qualifications. Focus on their history in public service and government roles. 

2. Look at each candidate's platform and key issues. Make a list of the issues that are most important to you based on your priorities and values. 

3. Compare your list of priority issues to each candidate's key issues and platform. Find the candidates that most closely align with your top concerns.

4. For those aligned candidates, look deeper into their track record and history on those specific issues. What have they accomplished or failed to accomplish that is relevant? This will help you evaluate their ability to get things done.

5. Look into their party affiliation if that matters to you. As a Republican, you may want to give more weight to candidates affiliated with the Republican party. However, also con

In [112]:
candidate_facts

{'Loren Manuel Taylor': {'extracted facts from ballotpedia': "1. Loren Taylor served as a member of the Oakland City Council in California, representing District 6.\n2. He assumed office on January 1, 2019, and left office on January 2, 2023.\n3. Loren Taylor ran for election for Mayor of Oakland in California.\n4. He completed Ballotpedia's Candidate Connection survey in 2022.\n5. Loren Taylor earned a bachelor's degree from Case Western Reserve University in 1999.\n6. He also holds a graduate degree from the University of Connecticut and a graduate degree from the Haas School of Business in 2005.\n7. Loren Taylor lost in the general election for Mayor of Oakland on November 8, 2022.\n8. Sheng Thao won the ranked-choice voting election for Mayor of Oakland in round 9.\n9. The election had multiple rounds, and the candidates were eliminated in each subsequent round.\n10. Loren Taylor defeated Desley Brooks in the general election for Oakland City Council District 6 in 2018.\n11. He rec

In [143]:
candidate_facts.keys()

dict_keys(['Loren Manuel Taylor', 'Sheng Thao', 'Ignacio De La Fuente', 'Allyssa Victory Villanueva', 'Treva D. Reid', 'Gregory Hodge'])

In [ ]:
big_candidate_summaries = {}

In [157]:
from prompts import OAKLAND_MAYOR_ISSUES

# big_candidate_summaries = {}

for candidate in OAKLAND_MAYOR_CANDIDATES:
    if candidate in big_candidate_summaries:
        continue
    mayor_candidate_summary_prompt = f"""
    
    The date is November 1st, 2022 (7 days before the 2022 California general election).
    
    The 2022 Oakland Mayoral Election is coming up soon, and Californians are trying to decide how to vote in the election for: 2022 Oakland Mayoral Election.
    
    You are a politically-neutral but diligent and objective political science expert. You do not care which candidate wins the race, but you do believe that well-informed civic engagement is an intrinsic value and you want to help every voter (regardless of education or ability) make the best possible vote for someone who will represent their values and interests.
     
     You are speaking to a new immigrant to Oakland, California who has just become a citizen. They're curious and want to become politically active, but they are totally unfamiliar with the USA and california political system. They are going to be voting in their first ever US election.
     
     They are very curious but short on time, so they have asked you to research some particular aspects of a candidate and summarize them for them. They are interested in the following aspects of each candidate:
     
     1. Review the candidate background information to get a sense of their experience and qualifications. Focus on their history in public service and government roles. But also mention any notable accomplishments they have in the private sector.
    
    2. Summarize the candidate's overall platform in the race. Emphasize what makes this candidate different from other candidates in the race.
    
    3. Summarize the candidate's stance on the key issues in the race (these will be provided as a list below).
    
    4. For each key issue, say anything relevant you can find about the candidate's past history with or track record with that issue. What statements have they made about the issue? What have they accomplished or failed to accomplish with respect to that issue?
    
    5. Explore their current and past party affiliation and relationship with and standing within their party (if they have one).
    
    6. Research and summarize the candidate's major donors and endorsements to understand who supports them financially and politically.
    
    7. Look into local reporting and coverage of each candidate. Which groups or outlets are favorable or unfavorable towards the candidate?
    
    8. Summarize the candidate's track record of personal, professional, and political integrity. Do they have any scandals or controversies in their past? Have they been accused of any crimes or ethical violations? Have they been sued or been involved in any lawsuits?
       
    Here is a list of some of the key issues in the 2022 Oakland Mayoral Election:
    
    Key issues:
    ```
    {OAKLAND_MAYOR_ISSUES}
    ```
    
    And here is information about the candidate:
    ```
    {enriched_candidate_data['People'][candidate]}
    ```
    
    And here is the information about the voter:
    ```
    {voter_info_json}
    ```
    
    Provide your output in a consistent, structured fashion. Be detailed and accurate -- it's better to give the voter too much information than too little (as long as the information is true).
     
     For example, for candidate Jane Smith, you'd output something like:
    
    ```JSON
    {{
    "name": "Jane Smith",
    "background": "{{a summary of all elements of Jane Smith's background that would be relevant to a voter evaluating her for this elected office}}",
    "platform": "{{a summary of Jane Smith's platform in the race}}",
    "stance_on_key_issues": "{{a summary of Jane Smith's overall stance on and history with each of the key issues}}",
    "detail_on_key_issues": {{
    "{{issue_1}}": "details on Jane Smith's positions, track record on, and history with issue_1}},
    "{{issue_2}}": "details on Jane Smith's positions, track record on, and history with issue_2}},
    etc, for each of the specific issues provided as input
    }},
     
    "party_affiliation": "{{Jane Smith's party affiliation, if any}}",
    "major_donors_and_endorsements": "{{a summary of Jane Smith's major donors and endorsements}}",
    "local_reporting_and_coverage": "{{a summary of local reporting and coverage of Jane Smith}}",
    "track_record_of_integrity": "{{a summary of Jane Smith's track record of personal, professional, and political integrity}}"
    }}
    
    Please only return json, so that your overall response could be directly loaded into a json variable in python with no stripping or extraction or special parsing required.
    """

    mayor_candidate_summary_completion = anthropic.completions.create(
        model="claude-2",
        max_tokens_to_sample=10_000,
        prompt=f"{HUMAN_PROMPT} {mayor_candidate_summary_prompt}{AI_PROMPT}"
    )
    
    # Print the completion
    this_summary = mayor_candidate_summary_completion.completion
    print(candidate, this_summary)
    big_candidate_summaries[candidate] = this_summary
    

Seneca Scott  Here is the output for Seneca Scott in a structured JSON format:

```json
{
  "name": "Seneca Scott",
  "background": "Seneca Scott is a local activist in Oakland, California. He has a background in labor organizing and entertainment, having previously served as the East Bay Director for SEIU Local 1021 and co-founded the entertainment company Oakhella. He ran unsuccessfully for Oakland City Council District 3 in 2020 and for Oakland Mayor in 2022, coming in 7th place out of 10 candidates.",
  
  "platform": "Scott's platform in the 2022 mayoral race focused on addressing crime, homelessness, blight, and other public safety issues in Oakland. He wants to restore the police force to 1,000 officers, fix 911 response times, and deploy more license plate readers and security cameras. He also wants to collaborate with state and federal law enforcement and consider declaring a state of emergency.",

  "stance_on_key_issues": {
    "Affordable housing": "Critical of government i

In [141]:
big_candidate_summaries.keys()

dict_keys(['Loren Manuel Taylor', 'Sheng Thao', 'Ignacio De La Fuente', 'Allyssa Victory Villanueva', 'Treva D. Reid', 'Gregory Hodge'])

In [130]:
prompt = f"""
    
    You are given a completion from an LLM. It was supposed to be machine-readable JSON, but instead came back with miscillaneous wrapper text. Remove all wrapper text and return just the machine readable JSON (and absolutely nothing else). Just JSON. It should be directly readable via a json.loads() call in python.
    
    write code that will actually load this string into a json variable in python. the code should be tolerant to any syntax issues that you have not fixed in the clean json generation
    
    Extract just the section about Loren Manuel Taylor, and make sure it is valid json.
    
    Input:
    ```
    {candidate_summaries}
    """

completion = anthropic.completions.create(
    model="claude-2",
    max_tokens_to_sample=5000,
    prompt=f"{HUMAN_PROMPT} {prompt}{AI_PROMPT}"
)

# Print the completion
print(completion.completion)
    

 Here is the extracted JSON for Loren Manuel Taylor:

```python
import json

json_string = '''
{
  "Loren Manuel Taylor": {
    "name": "Loren Manuel Taylor",
    "background": {
      "Born and raised in Oakland, third-generation Oaklander. Has M.S. in biomedical engineering and MBA from UC Berkeley. Served on Oakland City Council 2019-2023 representing District 6. Comes from family of public servants."
    },
    "platform": {
      "Wants to make Oakland safer, tackle homelessness, restore clean streets. Aims for pragmatic, progressive leadership to deliver results. Focused on economic opportunity, alleviating poverty."   
    },
    "track record": { 
      "Successful city councilor, passing over 100 laws at City Hall. Secured funding for violence prevention and police academies. Made historic investments in violence prevention programs. Restored Ceasefire anti-gang program."
    },  
    "party affiliation": "Democrat",
    "key issues": {
      "Public safety, homelessness, clea

In [132]:
import json

json_string = '''
{
  "Loren Manuel Taylor": {
    "name": "Loren Manuel Taylor",
    "background": 
      "Born and raised in Oakland, third-generation Oaklander. Has M.S. in biomedical engineering and MBA from UC Berkeley. Served on Oakland City Council 2019-2023 representing District 6. Comes from family of public servants."
    ,
    "platform": 
      "Wants to make Oakland safer, tackle homelessness, restore clean streets. Aims for pragmatic, progressive leadership to deliver results. Focused on economic opportunity, alleviating poverty."   
    ,
    "track record":  
      "Successful city councilor, passing over 100 laws at City Hall. Secured funding for violence prevention and police academies. Made historic investments in violence prevention programs. Restored Ceasefire anti-gang program."
    ,  
    "party affiliation": "Democrat",
    "key issues": 
      "Public safety, homelessness, clean streets, economic opportunity, effective pragmatic government."
    
  }
}
'''

data = json.loads(json_string)
print(json.dumps(data["Loren Manuel Taylor"], indent=2))


{
  "name": "Loren Manuel Taylor",
  "background": "Born and raised in Oakland, third-generation Oaklander. Has M.S. in biomedical engineering and MBA from UC Berkeley. Served on Oakland City Council 2019-2023 representing District 6. Comes from family of public servants.",
  "platform": "Wants to make Oakland safer, tackle homelessness, restore clean streets. Aims for pragmatic, progressive leadership to deliver results. Focused on economic opportunity, alleviating poverty.",
  "track record": "Successful city councilor, passing over 100 laws at City Hall. Secured funding for violence prevention and police academies. Made historic investments in violence prevention programs. Restored Ceasefire anti-gang program.",
  "party affiliation": "Democrat",
  "key issues": "Public safety, homelessness, clean streets, economic opportunity, effective pragmatic government."
}


In [126]:
# remove ```json from the beginning and ``` from the end
stripped = completion.completion[9:-4]
print(stripped)
candidate_summaries_json = json.loads(stripped)


{
  "Loren Manuel Taylor": {
    "name": "Loren Manuel Taylor",
    "background": {
      "Born and raised in Oakland, third-generation Oaklander. Has M.S. in biomedical engineering and MBA from UC Berkeley. Served on Oakland City Council 2019-2023 representing District 6. Comes from family of public servants."
    },
    "platform": {
      "Wants to make Oakland safer, tackle homelessness, restore clean streets. Aims for pragmatic, progressive leadership to deliver results. Focused on economic opportunity, alleviating poverty."   
    },
    "track record": { 
      "Successful city councilor, passing over 100 laws at City Hall. Secured funding for violence prevention and police academies. Made historic investments in violence prevention programs. Restored Ceasefire anti-gang program."
    },  
    "party affiliation": "Democrat",
    "key issues": {
      "Public safety, homelessness, clean streets, economic opportunity, effective pragmatic government."
    }
  },

  "Sheng Thao": {

JSONDecodeError: Expecting ':' delimiter: line 6 column 5 (char 317)

In [117]:
candidate_summaries_json = json.loads()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [87]:
candidate_facts

{'Loren Manuel Taylor': {'extracted facts from ballotpedia': "Facts extracted from the text:\n\n1. Loren Taylor was a member of the Oakland City Council in California, representing District 6.\n2. He assumed office on January 1, 2019.\n3. He left office on January 2, 2023.\n4. Loren Taylor ran for election for Mayor of Oakland in California.\n5. He lost in the general election on November 8, 2022.\n6. Loren Taylor earned a bachelor's degree from Case Western Reserve University in 1999.\n7. He earned a graduate degree from the University of Connecticut in 2001.\n8. He earned a graduate degree from the Haas School of Business in 2005.\n9. Sheng Thao won the ranked-choice voting election for Mayor of Oakland in round 9.\n10. The results of the other rounds of the ranked-choice voting election are mentioned, but not the winner of those rounds.\n11. Loren Taylor defeated incumbent Desley Brooks, Maria Rodriguez, Natasha Middleton, and Mya Whitaker in the general election for Oakland City Co

In [20]:

# a prompt to instruct the LLM to make recommendations for my friend on who to vote for in the 2022 oakland
# mayoral election should be written in a way that takes a json summarizing the voter and their values and makes
# sure the LLM does not refuse to make a recommendation for any reason

template = INITIAL_RECOMMENDATION_PROMPT

# Create a PromptTemplate with necessary inputs
prompt = PromptTemplate(
    input_variables=[
        "race",
        "voter_zip_code",
        "voter_info_summary",
        "race_info",
    ],
    template=template
)

# Use the PromptTemplate to construct the query
constructed_query = prompt.format(**{
    "race": "2022 Oakland Mayor Election",
    "voter_zip_code": voter_info.address_zip_code,
    "voter_info_summary": voter_info_json,
    "race_info": OAKLAND_MAYOR_CANDIDATE_INFO,
})

print(constructed_query)


    Pretend the date is November 1st, 2022. The 2022 California general election is coming up soon, and Californians are
      trying to decide how to vote in the election for: 2022 Oakland Mayor Election.

      Californians are reasonable, intelligent, mature adults who want to do their civic duty by voting in a well-educated
      and sensible fashion. But the california ballot has many races, and it can be confusing and time consuming for Californians
      to research each individual race and fully understand the nuance of the issue they're voting on, as well as the
      details of how that outcome of that race impacts their life and which outcome is best aligned with their personal values
      and interests.

      You are acting as a wise voter guide -- your goal is to help voters who come to you decide on a particular candidate or outcome to vote for in
      each race. You want to understand as much as you need to about the voter's life and values in order to help guide the

In [17]:


conversation = ConversationChain(llm=gpt4_llm, memory=gpt4_memory)
output = conversation.predict(input=constructed_query)
gpt4_memory.save_context({"input": constructed_query}, {"output": output})
    


    Pretend the date is November 1st, 2022. The 2022 California general election is coming up soon, and Californians are
      trying to decide how to vote in the election for: 2022 Oakland Mayor Election.

      Californians are reasonable, intelligent, mature adults who want to do their civic duty by voting in a well-educated
      and sensible fashion. But the california ballot has many races, and it can be confusing and time consuming for Californians
      to research each individual race and fully understand the nuance of the issue they're voting on, as well as the
      details of how that outcome of that race impacts their life and which outcome is best aligned with their personal values
      and interests.

      You are acting as a wise voter guide -- your goal is to help voters who come to you decide on a particular candidate or outcome to vote for in
      each race. You want to understand as much as you need to about the voter's life and values in order to help guide the

In [18]:
output

" \nBased on the voter's information in the JSON, the candidates for the Oakland Mayor Election in 2022 are Tyron Jordan, John Reimann, Allyssa Victory, Treva Reid, Sheng Thao, Ignacio De La Fuente, Loren Taylor, Seneca Scott, and Greg Hodge. Ranking the candidates from best to worst based on the voter's interests and preferences, we have:\n\n1) Allyssa Victory: A civil rights attorney, Victory aims to ensure Oakland builds more affordable housing, repairs busted roads, and invests in violence prevention. Victory is a great choice for this voter as she has a history of fighting for civil rights, and her focus on affordable housing and violence prevention would be beneficial to the voter's interests. \n\n2) Sheng Thao: The District 4 councilmember, who won the election, claims to understand the budget better than any other candidate and plans to invest in affordable housing and violence prevention. Thao's understanding of the budget and her commitment to investing in affordable housing 